Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/deployment/deploy-to-local/register-model-deploy-local-advanced.png)

# Register model and deploy locally with advanced usages

This example shows how to deploy a web service in step-by-step fashion:

 1. Register model
 2. Deploy the image as a web service in a local Docker container.
 3. Quickly test changes to your entry script by reloading the local service.
 4. Optionally, you can also make changes to model, conda or extra_docker_file_steps and update local service

## Prerequisites
If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you go through the [configuration](../../../configuration.ipynb) Notebook first if you haven't.

In [ ]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

## Initialize Workspace

Initialize a workspace object from persisted configuration.

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

## Register Model

You can add tags and descriptions to your models. we are using `sklearn_regression_model.pkl` file in the current directory as a model with the name `sklearn_regression_model_local_adv` in the workspace.

Using tags, you can track useful information such as the name and version of the machine learning library used to train the model, framework, category, target customer etc. Note that tags must be alphanumeric.

In [ ]:
from azureml.core.model import Model

model = Model.register(model_path="sklearn_regression_model.pkl",
                       model_name="sklearn_regression_model_local_adv",
                       tags={'area': "diabetes", 'type': "regression"},
                       description="Ridge regression model to predict diabetes",
                       workspace=ws)

Manage your dependencies in a folder

In [ ]:
import os

source_directory = "C:/abc"

os.makedirs(source_directory, exist_ok=True)
os.makedirs("C:/abc/x/y", exist_ok=True)
os.makedirs("C:/abc/env", exist_ok=True)
os.makedirs("C:/abc/dockerstep", exist_ok=True)

Show `score.py`. 

In [ ]:
%%writefile C:/abc/x/y/score.py
import os
import pickle
import json
import numpy as np
from sklearn.externals import joblib
from sklearn.linear_model import Ridge
from azureml.core.model import Model

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType

def init():
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment. Join this path with the filename of the model file.
    # It holds the path to the directory that contains the deployed model (./azureml-models/$MODEL_NAME/$VERSION)
    # If there are multiple models, this value is the path to the directory containing all deployed models (./azureml-models)
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'sklearn_regression_model.pkl')
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)
    global name
    # note here, entire source directory on inference config gets added into image
    # bellow is the example how you can use any extra files in image
    with open('./abc/extradata.json') as json_file:  
        data = json.load(json_file)
        name = data["people"][0]["name"]

input_sample = np.array([[10,9,8,7,6,5,4,3,2,1]])
output_sample = np.array([3726.995])

@input_schema('data', NumpyParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))
def run(data):
    try:
        result = model.predict(data)
        # you can return any datatype as long as it is JSON-serializable
        return "Hello " + name + " here is your result = " + str(result)
    except Exception as e:
        error = str(e)
        return error

In [ ]:
%%writefile C:/abc/env/myenv.yml
name: project_environment
dependencies:
  - python=3.6.2
  - pip:
    - azureml-defaults
    - scikit-learn
    - numpy
    - inference-schema[numpy-support]

In [ ]:
%%writefile C:/abc/dockerstep/customDockerStep.txt
RUN echo "this is test"

In [ ]:
%%writefile C:/abc/extradata.json
{
    "people": [
        {
            "website": "microsoft.com", 
            "from": "Seattle", 
            "name": "Mrudula"
        }
    ]
}

## Create Inference Configuration

 - source_directory = holds source path as string, this entire folder gets added in image so its really easy to access any files within this folder or subfolder
 - runtime = Which runtime to use for the image. Current supported runtimes are 'spark-py' and 'python
 - entry_script = contains logic specific to initializing your model and running predictions
 - conda_file = manages conda and python package dependencies.
 - extra_docker_file_steps = optional: any extra steps you want to inject into docker file

In [ ]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(source_directory="C:/abc",
                                   runtime="python", 
                                   entry_script="x/y/score.py",
                                   conda_file="env/myenv.yml", 
                                   extra_docker_file_steps="dockerstep/customDockerStep.txt")

## Deploy Model as a Local Docker Web Service

*Make sure you have Docker installed and running.*

Note that the service creation can take few minutes.

NOTE:

The Docker image runs as a Linux container. If you are running Docker for Windows, you need to ensure the Linux Engine is running:

    # PowerShell command to switch to Linux engine
    & 'C:\Program Files\Docker\Docker\DockerCli.exe' -SwitchLinuxEngine

In [ ]:
from azureml.core.webservice import LocalWebservice

# This is optional, if not provided Docker will choose a random unused port.
deployment_config = LocalWebservice.deploy_configuration(port=6789)

local_service = Model.deploy(ws, "test", [model], inference_config, deployment_config)

local_service.wait_for_deployment()

In [ ]:
print('Local service port: {}'.format(local_service.port))

## Check Status and Get Container Logs


In [ ]:
print(local_service.get_logs())

## Test Web Service

Call the web service with some input data to get a prediction.

In [ ]:
import json

sample_input = json.dumps({
    'data': [
        [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
    ]
})

sample_input = bytes(sample_input, encoding='utf-8')

print(local_service.run(input_data=sample_input))

## Reload Service

You can update your score.py file and then call `reload()` to quickly restart the service. This will only reload your execution script and dependency files, it will not rebuild the underlying Docker image. As a result, `reload()` is fast, but if you do need to rebuild the image -- to add a new Conda or pip package, for instance -- you will have to call `update()`, instead (see below).

In [ ]:
%%writefile C:/abc/x/y/score.py
import os
import pickle
import json
import numpy as np
from sklearn.externals import joblib
from sklearn.linear_model import Ridge

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType

def init():
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # For multiple models, it points to the folder containing all deployed models (./azureml-models)
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'sklearn_regression_model.pkl')
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)
    global name, from_location
    # note here, entire source directory on inference config gets added into image
    # bellow is the example how you can use any extra files in image
    with open('./abc/extradata.json') as json_file:  
        data = json.load(json_file)
        name = data["people"][0]["name"]
        from_location = data["people"][0]["from"]

input_sample = np.array([[10,9,8,7,6,5,4,3,2,1]])
output_sample = np.array([3726.995])

@input_schema('data', NumpyParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))
def run(data):
    try:
        result = model.predict(data)
        # you can return any datatype as long as it is JSON-serializable
        return "Hello  " + name + " from " + from_location + " here is your result = " + str(result)
    except Exception as e:
        error = str(e)
        return error

In [ ]:
local_service.reload()
print("--------------------------------------------------------------")

# After calling reload(), run() will return the updated message.
local_service.run(input_data=sample_input)

## Update Service

If you want to change your model(s), Conda dependencies, or deployment configuration, call `update()` to rebuild the Docker image.

```python

local_service.update(models=[SomeOtherModelObject],
                     deployment_config=local_config,
                     inference_config=inference_config)
```

## Delete Service

In [ ]:
local_service.delete()